In [1]:
using BSON, Dates, DelimitedFiles, Downloads, Flux, Printf
include("neural.jl");

[ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]


In [2]:
function generate_inout(ρ_profiles, c1_profiles, ϕ_profiles; window_width, dx)
    window_bins = 2 * round(Int, window_width / dx) + 1
    ρ_windows_all = Vector{Vector{Float32}}()
    c1_values_all = Vector{Float32}()
    ϕ_functions_all = Vector{Vector{Float32}}()
    for (ρ, c1, ϕ) in zip(ρ_profiles, c1_profiles, ϕ_profiles)
        ρ_windows = generate_windows(ρ; window_bins)
        ϕ_func = generate_phi(ϕ,ρ)
        s = 40 
        for i in collect(1:s:length(c1)) 
            if !isfinite(c1[i])
                continue
            end
            push!(ρ_windows_all, ρ_windows[:,i])
            push!(c1_values_all, c1[i])
            push!(ϕ_functions_all, ϕ_func[:,i])
            
            #push!(ρ_windows_all, reverse(ρ_windows[:,i]))
            #push!(c1_values_all, c1[i])
            #push!(ϕ_functions_all, ϕ_func[:,i])
            
        end
    end
    reduce(hcat, ρ_windows_all), c1_values_all', reduce(hcat, ϕ_functions_all)
end

generate_inout (generic function with 1 method)

In [3]:
Downloads.download("https://myfiles.uni-bayreuth.de/filr/public-link/file-download/043490a392e1752f01931b17e3711848/93318/1417842602830201006/Simulations.tar", "Simulations.tar")
run(`tar xf Simulations.tar`)
datadir = "Simulations"
ρ_profiles, c1_profiles, ϕ_profiles = read_sim_data(datadir);
ρ_windows, c1_values, ϕ_functions = generate_inout(ρ_profiles, c1_profiles, ϕ_profiles; window_width=2.0, dx=0.01)
size(ρ_windows), size(c1_values), size(ϕ_functions)

((401, 270949), (1, 270949), (150, 270949))

In [4]:
using CUDA
ρ_windows, c1_values, ϕ_functions = (ρ_windows, c1_values, ϕ_functions) |> gpu

model = Chain(
    Dense(size(vcat(ρ_windows, ϕ_functions))[1] => 512, softplus),
    Dense(512 => 256, softplus),
    Dense(256 => 128, softplus),
    Dense(128 => 64, softplus),
    Dense(64 => 32, softplus),
    Dense(32 => 1) 
) |> gpu

display(model)  

opt = Flux.setup(Adam(), model) 

loader = Flux.DataLoader((vcat(ρ_windows, ϕ_functions), c1_values), batchsize=256, shuffle=true)|> gpu  

loss(m, x, y) = Flux.mse(m(x), y)  
metric(m, x, y) = Flux.mae(m(x), y)  
get_learning_rate(epoch; initial=0.0001, rate=0.03, wait=5) = epoch < wait ? initial : initial * (1 - rate)^(epoch - wait)

model_savefile = "model_test.bson"
println("Saving model to $(model_savefile)")
for epoch in 1:150 
    learning_rate = get_learning_rate(epoch)
    Flux.adjust!(opt, learning_rate)      
    @printf "Epoch: %3i (learning_rate: %.2e)..." epoch learning_rate; flush(stdout)
    Flux.train!(loss, model, loader, opt) 
    @printf " loss: %.5f, metric: %.5f\n" loss(model, vcat(ρ_windows, ϕ_functions), c1_values) metric(model, vcat(ρ_windows, ϕ_functions), c1_values); flush(stdout)
    model = model |> cpu
    BSON.@save model_savefile model
    model = model |> gpu
end

model

Chain(
  Dense(551 => 512, softplus),          # 282_624 parameters
  Dense(512 => 256, softplus),          # 131_328 parameters
  Dense(256 => 128, softplus),          # 32_896 parameters
  Dense(128 => 64, softplus),           # 8_256 parameters
  Dense(64 => 32, softplus),            # 2_080 parameters
  Dense(32 => 1),                       # 33 parameters
)                   # Total: 12 arrays, 457_217 parameters, 1.656 KiB.

Saving model to model_test.bson
Epoch:   1 (learning_rate: 1.00e-04)... loss: 0.23144, metric: 0.18973
Epoch:   2 (learning_rate: 1.00e-04)... loss: 0.13739, metric: 0.17833
Epoch:   3 (learning_rate: 1.00e-04)... loss: 0.08051, metric: 0.14319
Epoch:   4 (learning_rate: 1.00e-04)... loss: 0.09872, metric: 0.13872
Epoch:   5 (learning_rate: 1.00e-04)... loss: 0.07621, metric: 0.12498
Epoch:   6 (learning_rate: 9.70e-05)... loss: 0.05497, metric: 0.11087
Epoch:   7 (learning_rate: 9.41e-05)... loss: 0.05415, metric: 0.12007
Epoch:   8 (learning_rate: 9.13e-05)...

LoadError: InterruptException: